# Config Creator

In [2]:
#C:\Users\CW3\BMS Group\BMS Re - Documents\Actuarial\Models\New Model Development\Blank Analysis Template\Code\ReinsuranceStrategy_ConfigSpecsv2.xlsm
#C:\Users\CW3\BMS Group\BMS Re - Documents\Actuarial\Individual Actuary Folders\Clare Whitlam\Work With Miles\Code\ReinsuranceStrategy_ConfigSpecsv2.xlsm
#C:\Users\CW3\BMS Group\BMS Re - Documents\Actuarial\Individual Actuary Folders\Clare Whitlam\Strategy - working\ReinsuranceStrategy_ConfigSpecsv2.xlsm

In [1]:
# Purpose: Create config file based on current version of Excel spec format
import os
import pandas as pd
import polars as pl
import xlwings as xw
import configparser

config = configparser.ConfigParser()     

def load_spec_table_to_df(xl_sht, table_name):
    return  (book.sheets[xl_sht]
            .range(table_name + "[[#All]]")
            .options(pd.DataFrame, index=False,header=True)
            .value.astype(str))

def getColList(colstring):
    if "," in colstring:
        return [x.strip() for x in colstring.split(",")]        
    else:
        return [colstring.strip()]
    
def getKeys(collist,x):
    # print(collist)
    # print(x)
    if len(collist)>1:
        return '|'.join(x[collist].values.tolist())
    else:
        return x[collist[0]]

def getVals(collist,x):
    if len(collist)>1:
        result=x[collist].values.tolist()
        result=tuple(result)
    else:
        result=x[collist[0]]
    return result
    
if __name__ == "__main__":
    print('Enter full path and filename for analysis file')
    x=input()
    x=x.replace(os.sep,"/")
    xw.Book(x).set_mock_caller()
    
    book = xw.Book.caller()
    filepath=book.fullname[:len(book.fullname)-len(book.name)].replace(os.sep,"/")  
    
    #Import table of dictionaries that contains all specs
    tblDictSpecs=load_spec_table_to_df("Python Dictionaries and dtypes","Tbl_dictSpecs")

    for row1 in tblDictSpecs.iterrows():
        print(row1[1]['Dictionary Variable'])
        tempSourceTbl=load_spec_table_to_df("Python Dictionaries and dtypes",row1[1]['Dictionary Source Table'])
        tempListKeyCols=getColList(row1[1]['Key Column (s)'])
        tempListValCols=getColList(row1[1]['Value Column (s)'])
        tempSourceTbl['keys']=tempSourceTbl.apply(lambda x: getKeys(tempListKeyCols,x),axis=1)
        tempSourceTbl['vals']=tempSourceTbl.apply(lambda x: getVals(tempListValCols,x),axis=1)     
        config[row1[1]['Dictionary Variable']]=dict(zip(tempSourceTbl["keys"],tempSourceTbl["vals"]))
        config[row1[1]['Dictionary Variable']+'_keys']=dict(zip(tempSourceTbl["keys"],tempSourceTbl["keys"]))
        tempAddToPreppedSpecs="True" if row1[1]['Add to Prepped Specs']=="True" else "False"
        config[row1[1]['Dictionary Variable']]['addtospecs']=tempAddToPreppedSpecs

    config['dict_keyCols']=dict(zip(tblDictSpecs['Dictionary Variable'],tblDictSpecs['Key Column (s)']))
    config['dict_keyCols_keys']=dict(zip(tblDictSpecs['Dictionary Variable'],tblDictSpecs['Dictionary Variable']))
    config['dict_valCols']=dict(zip(tblDictSpecs['Dictionary Variable'],tblDictSpecs['Value Column (s)']))
    config['dict_valCols_keys']=dict(zip(tblDictSpecs['Dictionary Variable'],tblDictSpecs['Dictionary Variable']))

    with open(filepath+'config.ini', 'w') as configfile:
        config.write(configfile)

Enter full path and filename for analysis file
dict_specSheetName
dict_specTableName
dict_specHeader
dict_specInfoTable
dict_excelResultSheets
dict_excelResultTables
dict_excelResultFunctions
dict_userPaths
dict_specDataFormats
dict_panelMapWidgetTypes
dict_panelMainMenu
dict_panelMainMenuActions
dict_panelWidgets
dict_panelWidgetGroups
dict_panelWidgetGroupsTiming
dict_panelWidgetGroupMembers
dict_panelWidgetSpecs
dict_panelTabs
dict_panelDataforms
dict_panelDataformCards
dict_panelCustomDataform
dict_panelDataTypes
dict_complement


In [6]:
pip install xlwings


     ---------------------------------------- 1.7/1.7 MB 13.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\CW3\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
#THIS IS CODE TO IMPORT CONFIG INTO DICTIONARY IN PROJECT

import configparser
config = configparser.ConfigParser()
config.read("C:/Users/CW3/BMS Group/BMS Re - Documents/Actuarial/Models/New Model Development/Reinsurance Strategy Analysis/Config.ini")
print(config)
dict_dataTypes={}

def configparser_to_dict(config: configparser.ConfigParser) -> dict:
    config_dict = {}
    for section in config.sections():
        config_dict[section] = {}
        if section!='dict_dataTypes':
            for key, value in config.items(section):
            # Now try to convert back to original types if possible
                for boolean in ['True', 'False', 'None']:
                    if value == boolean:
                        value = bool(boolean)

                # Try to convert to float or int
                try:
                    if isinstance(value, str):
                        if '.' in value:
                            value = float(value)
                        else:
                            value = int(value)
                except ValueError:
                    pass

                config_dict[section][key] = value
        else:
            config_dict[section]=dict(config.items(section))

    # Now drop root section if present
    config_dict.pop('root', None)
    return config_dict

# config_dict=configparser_to_dict(config)
# for key in config_dict.keys():
#     exec(key+"=config_dict[key]")


In [5]:
dict_userpaths

{'cw3': 'C:\\Users\\CW3\\BMS Group\\BMS Re - Documents\\Actuarial\\Models\\New Model Development\\Reinsurance Strategy Analysis\\Reinsurance Strategy Analyses',
 'm8c': 'C:\\Users\\M8C\\OneDrive - BMS Group\\Documents\\VsCode\\Reinsurance Strategy Analyses'}